In [11]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.datasets.boston_housing import load_data
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import KFold

In [5]:
(x_train, y_train), (x_test, y_test) = load_data(path='boston_housing.npz', test_split=0.2, seed=777)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(404, 13) (404,)
(102, 13) (102,)


# Data Regularization

In [6]:
mean = np.mean(x_train, axis=0)
std  = np.std(x_train, axis=0)

x_train = (x_train - mean) / std
x_test  = (x_test - mean) / std

In [7]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.33, random_state=777)

# Modeling

In [8]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(13, )))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))
model.compile(
    optimizer='adam', 
    loss='mse',
    metrics=['mae'])

In [9]:
history = model.fit(x_train, y_train, epochs=300, validation_data = (x_val, y_val))

Epoch 1/300
9/9 [==============================] - 1s 18ms/step - loss: 548.3932 - mae: 21.7612 - val_loss: 596.2660 - val_mae: 22.2070
Epoch 2/300
9/9 [==============================] - 0s 8ms/step - loss: 520.7364 - mae: 21.0964 - val_loss: 566.8430 - val_mae: 21.5211
Epoch 3/300
9/9 [==============================] - 0s 7ms/step - loss: 492.6245 - mae: 20.3983 - val_loss: 534.4087 - val_mae: 20.7482
Epoch 4/300
9/9 [==============================] - 0s 8ms/step - loss: 460.2799 - mae: 19.5886 - val_loss: 495.5399 - val_mae: 19.8176
Epoch 5/300
9/9 [==============================] - 0s 7ms/step - loss: 420.2807 - mae: 18.6061 - val_loss: 449.0143 - val_mae: 18.6682
Epoch 6/300
9/9 [==============================] - 0s 7ms/step - loss: 375.1156 - mae: 17.4244 - val_loss: 392.7588 - val_mae: 17.2136
Epoch 7/300
9/9 [==============================] - 0s 8ms/step - loss: 321.0923 - mae: 15.9684 - val_loss: 330.1483 - val_mae: 15.4766
Epoch 8/300
9/9 [==============================] - 0s 

In [10]:
model.evaluate(x_test, y_test)

4/4 [==============================] - 0s 7ms/step - loss: 8.9855 - mae: 2.1708


[8.98548698425293, 2.1707615852355957]

# K-fold

In [15]:
k = 3
kfold = KFold(n_splits=k, random_state=777, shuffle=True)

def get_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(13, )))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(
        optimizer='adam', 
        loss='mse',
        metrics=['mae'])
    
    return model

mae_list = []

for train_index, val_index in kfold.split(x_train):
    x_train_fold, x_val_fold = x_train[train_index], x_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    
    model = get_model()
    
    model.fit(x_train_fold, y_train_fold, epochs=300, validation_data = (x_val_fold, y_val_fold))
    _, test_mae = model.evaluate(x_test, y_test)
    mae_list.append(test_mae)

Epoch 1/300
6/6 [==============================] - 0s 20ms/step - loss: 536.3217 - mae: 21.3607 - val_loss: 512.0140 - val_mae: 20.9094
Epoch 2/300
6/6 [==============================] - 0s 9ms/step - loss: 516.9844 - mae: 20.8725 - val_loss: 492.1320 - val_mae: 20.3850
Epoch 3/300
6/6 [==============================] - 0s 8ms/step - loss: 495.9945 - mae: 20.3484 - val_loss: 471.3393 - val_mae: 19.8208
Epoch 4/300
6/6 [==============================] - 0s 9ms/step - loss: 474.1786 - mae: 19.7738 - val_loss: 448.5023 - val_mae: 19.1964
Epoch 5/300
6/6 [==============================] - 0s 8ms/step - loss: 449.5520 - mae: 19.1301 - val_loss: 423.2047 - val_mae: 18.4955
Epoch 6/300
6/6 [==============================] - 0s 7ms/step - loss: 422.8522 - mae: 18.4473 - val_loss: 394.6799 - val_mae: 17.6834
Epoch 7/300
6/6 [==============================] - 0s 9ms/step - loss: 393.2975 - mae: 17.6600 - val_loss: 362.6422 - val_mae: 16.7896
Epoch 8/300
6/6 [==============================] - 0s 

In [16]:
print(mae_list)

[2.3324081897735596, 2.3472378253936768, 2.389760971069336]


In [17]:
print(np.mean(mae_list))

2.356468995412191
